In [1]:
import torch
import numpy as np

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
features.shape

torch.Size([1000, 2])

In [4]:
print(features)

tensor([[-0.1826, -1.5024],
        [-0.3939,  0.5430],
        [ 0.9596, -1.0020],
        ...,
        [-1.1770, -0.7149],
        [ 0.9407, -0.5876],
        [ 0.7994, -0.9287]])


In [5]:
import torch.utils.data as Data

In [6]:
batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [7]:
for X,y in data_iter:
    print(X,y)
    break

tensor([[ 1.7978, -0.3469],
        [-1.4149, -2.2841],
        [ 0.6582, -0.1413],
        [-0.3356, -1.0223],
        [-0.7021,  1.7482],
        [ 0.0809, -0.7359],
        [-0.6911, -0.9840],
        [-0.7540,  2.1416],
        [ 0.0654, -1.2865],
        [ 1.0336,  0.1100]]) tensor([ 8.9558,  9.1297,  6.0069,  6.9972, -3.1353,  6.8670,  6.1906, -4.5895,
         8.7073,  5.8951])


In [9]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [11]:
loss = nn.MSELoss()
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
net = net.to("cuda:0")
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        X = X.to("cuda:0")
        y = y.to("cuda:0")
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000101
epoch 2, loss: 0.000069
epoch 3, loss: 0.000140


In [16]:
print(net.parameters())

<generator object Module.parameters at 0x000002006A343580>
